<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/Fraud/Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
db = pd.read_csv('/content/drive/MyDrive/AIEngineer/estimator-med-cost/ราคากลางของบิลแต่ละประเภทในแต่ละโรค.csv')
db = db[['simb_billing_code','local_billing_name','PP','EC','OT','CM','CP','mean']]

In [ ]:
db

In [ ]:
!pip -q install gradio pandas

In [7]:
import pandas as pd
import gradio as gr

In [44]:
# ฟังก์ชันสำหรับการค้นหาข้อมูลจากฐานข้อมูล
def search_medical_cost(input_json):
    diagnosis_list = eval(input_json)
    query = create_query(diagnosis_list)

    search_results = []
    for index, row in db.iterrows():
        match = True
        for dx_type, icd10_code in query.items():
            if icd10_code == "-":
                if row[dx_type] != "-":
                    match = False
                    break
            else:
                if not any(code in row[dx_type] for code in icd10_code):
                    match = False
                    break

        if match:
            search_results.append(row)

    return pd.DataFrame(search_results)

def create_query(real_icd10):
    query = {"PP": "-", 'EC': '-', 'OT': '-', 'CM': '-', 'CP': '-'}

    for item in real_icd10:
        dx_type = item['dx_type']
        icd10_code = item['icd10']
        if dx_type in query:
            if query[dx_type] == "-":
                query[dx_type] = [icd10_code]
            elif isinstance(query[dx_type], list):
                if icd10_code not in query[dx_type]:
                    query[dx_type].append(icd10_code)
            else:
                query[dx_type] = [query[dx_type], icd10_code]
    return query

def summarize(selected_rows, total_sum):
    if selected_rows is None or len(selected_rows) == 0:
        return "No rows selected.", total_sum
    total_sum += selected_rows["mean"].sum()
    return f"Selected rows added. Current total: {total_sum:.2f} THB", total_sum

# Gradio UI Setup
with gr.Blocks() as demo:
    gr.Markdown("### Medical Cost Estimation Demo 📟")

    input_json = gr.Textbox(
        label="Enter Diagnosis JSON",
        placeholder='[{"icd10": "S500", "dx_name": "Contusion of elbow", "dx_type": "PP"}, ...]'
    )
    search_button = gr.Button("Search")
    output_table = gr.DataFrame(
        headers=["simb_billing_code", "local_billing_name", "PP", "EC", "OT", "CM", "CP", "mean"],
        interactive=True
    )
    summary_text = gr.Textbox(label="Summary", interactive=False)
    def process_input(input_json):
        search_results_df = search_medical_cost(input_json)
        return search_results_df, "Review the table and select rows for summarization."

    total_sum = 0
    search_button.click(process_input, inputs=[input_json], outputs=[output_table, summary_text])
demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://21103cc055b3b60377.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2045, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1592, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://21103cc055b3b60377.gradio.live


In [ ]:
[{"icd10": "S500", "dx_name": "Contusion of elbow", "dx_type": "PP"}, {"icd10": "W200", "dx_name": "Struck by thrown, projected or falling object", "dx_type": "EC"}, {"icd10": "S300", "dx_name": "Contusion of lower back and pelvis", "dx_type": "OT"}, {"icd10": "S400", "dx_name": "Contusion of shoulder and upper arm", "dx_type": "OT"}]